In [ ]:
import folium as fl
import pandas as pd
import geopandas as gpd

In [ ]:
import sys

## Geocoding

In [ ]:
from geopy.geocoders import Nominatim

def geocode(city):
    geolocator = Nominatim(user_agent="ironhack")
    location = geolocator.geocode(city)
    latitude = location.latitude
    longitude = location.longitude
    return (latitude, longitude)
    


In [ ]:
city = "Girona, Spain"
coords = geocode(city)
coords

In [ ]:
city = "Cañas, Costa Rica"
coords = geocode(city)
coords

In [ ]:
fl.Map(location=coords)

In [ ]:
m = fl.Map(location=coords)
marker = fl.Marker(location=coords, draggable=False)
marker.add_to(m)
m

## Choropleth

In [ ]:
coords = [40.409120, -3.700144]

In [ ]:
mapa = gpd.read_file('../data/provincias.geojson')
natalidad = pd.read_csv('../data/natalidad.csv')

In [ ]:
m = fl.Map(location=coords, zoom_start=6)

In [ ]:
municipios_file = '../data/divisions-administratives-v2r1-municipis-1000000-20230511.json'
municipios = gpd.read_file(municipios_file)
municipios

In [ ]:
import numpy as np

In [ ]:
datos = municipios[['CODIMUNI']]
datos['valor'] = np.random.random(len(municipios))

In [ ]:
datos

In [ ]:

fl.Choropleth(
    geo_data=municipios,
    name='choropleth',
    data=datos,
    columns=['CODIMUNI', 'valor'],
    key_on='feature.properties.CODIMUNI',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Tasa de natalidad (%)'
).add_to(m)
display(m)

In [ ]:
municipios = pd.read_csv('data/municipios.csv')
municipios.Longitud = capitales.Longitud.str.replace(',','.').astype('float')
municipios.Latitud = capitales.Latitud.str.replace(',','.').astype('float')
capitales = municipios.get(municipios.Capital=='Si')

In [ ]:
municipios.sort_values('AREAM5000', ascending=False).head(10)

In [ ]:
selected = municipios.sort_values('AREAM5000', ascending=False).head(10)

In [ ]:
m = fl.Map(location=[40.409120, -3.700144], zoom_start=6)
for _,row in selected.iterrows():
    fl.Marker([row.geometry.centroid.y, row.geometry.centroid.x], popup=row['AREAM5000']).add_to(m)
    
for _, row in selected.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(row["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = fl.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "orange"})
    geo_j.add_to(m)
display(m)

https://stacindex.org/catalogs/catalonia-monthly-sentinel2#/?t=1

In [ ]:
import rasterio as rio


img_file = "../img/ortoimatge-sentinel2-mensual-v1r0-rgb8b-202305.tif"


In [ ]:
with rio.open(img_file) as src:
    img = src.read()
    src_crs = src.crs['init'].upper()
    min_lon, min_lat, max_lon, max_lat = src.bounds

In [ ]:
src_crs

In [ ]:
dst_crs = 'EPSG:4326'

In [ ]:
src.bounds

In [ ]:
from rasterio.enums import Resampling

upscale_factor = 1/4

with rio.open(img_file) as dataset:

    # resample data to target shape
    img = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height * upscale_factor),
            int(dataset.width * upscale_factor)
        ),
        resampling=Resampling.bilinear
    )

    # scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]),
        (dataset.height / data.shape[-2])
    )

    src_crs = dataset.crs['init'].upper()
    min_lon, min_lat, max_lon, max_lat = dataset.bounds

In [ ]:
img.shape

In [ ]:
transform

In [ ]:
from rasterio.plot import show


In [ ]:
## Conversion from UTM to WGS84 CRS

from pyproj import Transformer 

bounds_orig = [[min_lat, min_lon], [max_lat, max_lon]]

bounds_fin = []
 
for item in bounds_orig:   
    #converting to lat/lon
    lat = item[0]
    lon = item[1]
    
    proj = Transformer.from_crs(int(src_crs.split(":")[1]), int(dst_crs.split(":")[1]), always_xy=True)

    lon_n, lat_n = proj.transform(lon, lat)
    
    bounds_fin.append([lat_n, lon_n])

# Finding the centre latitude & longitude    
centre_lon = bounds_fin[0][1] + (bounds_fin[1][1] - bounds_fin[0][1])/2
centre_lat = bounds_fin[0][0] + (bounds_fin[1][0] - bounds_fin[0][0])/2

In [ ]:
bounds_fin

In [ ]:
import numpy as np

In [ ]:
m = fl.Map(location=[centre_lat, centre_lon], zoom_start = 6)

# img.transpose(1, 2, 0)
# np.moveaxis(img, 0, -1)

# Overlay raster using add_child() function
m.add_child(fl.raster_layers.ImageOverlay(np.moveaxis(img, 0, -1), 
                                          opacity=1, 
                                          bounds = bounds_fin)
           )

# Display map 
m

In [ ]:
m = fl.Map(location=[40.409120, -3.700144], zoom_start=6)

image = fl.raster_layers.ImageOverlay(
        name="Sentinel",
        image = img, 
        bounds=[[25, -180], [80, 180]],
        interactive=True,
        zindex=1,
    )

image.add_to(m) 